In [1]:
# JUST RUN ALL CELLS AND CHECK THE RESULTS AT THE BOTTOM

import argparse
import csv
import os
import shutil
from tqdm import tqdm

try: import setGPU
except ImportError: pass

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import numpy as np
import numpy.random as npr

import sys
import time
sys.path.insert(0,'/home/alvin/Documents/cs159/CS159Project/qpth/')

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('bmh')

import models

import sys

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

batchSz = 64

In [2]:
model = torch.load("work/mnist.lenet-optnet.nHidden:50.nineq:100.eps:0.0001/latest.pth", map_location=torch.device('cpu'))

In [3]:
testLoader = torch.utils.data.DataLoader(
    dset.MNIST('data/mnist', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batchSz, shuffle=False, **{})

In [20]:
def test(net):
    net.eval()
    test_loss = 0
    incorrect = 0

    start_time = time.time()
    with torch.no_grad():
        i = 0
        for data, target in testLoader:
            i += 1
            print('Testing model: {}/{}'.format(i, 10), end='\r')
            data, target = Variable(data), Variable(target)
            output = net(data)
            test_loss += F.nll_loss(output, target).data
            pred = output.data.max(1)[1] # get the index of the max log-probability
            incorrect += pred.ne(target.data).cpu().sum()

            if i >= 10:
                break
    
    dt = time.time() - start_time
    test_loss = test_loss
    test_loss /= 10 # loss function already averages over batch size
    nTotal = 10 * batchSz
    err = 100.*incorrect/nTotal
    print('\nTest set: Average loss: {:.4f}, Error: {}/{} ({:.0f}%)\n'.format(
        test_loss, incorrect, nTotal, err))
    print(f'Average time per batch: {dt / 10}')

In [21]:
model1 = models.LenetDynamic(nineq=100)

params = model.named_parameters()
params1 = model1.named_parameters()

dict_params1 = dict(params1)

for name, param in params:
    if name in dict_params1:
        dict_params1[name].data.copy_(param.data)

In [22]:
# OPTNET PERFORMANCE TEST
test(model)

Testing model: 1/10
--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------

Testing model: 2/10
--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------



In [23]:
# DYNAMIC PERFORMANCE TEST
test(model1)

Testing model: 10/10
Test set: Average loss: nan, Error: 584/640 (91%)

Average time per batch: 0.02909233570098877
